# Importing Libraries

In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
spark=SparkSession.builder.appName('TechTFQ').getOrCreate()


#Creating Dataframes  

In [0]:
df_subject = spark.read.csv('dbfs:/FileStore/shared_uploads/kumaranonymous77@gmail.com/subject.csv', header=True, inferSchema=True)

df_work = spark.read.csv('dbfs:/FileStore/shared_uploads/kumaranonymous77@gmail.com/work-1.csv', header=True, inferSchema=True)

df_artist = spark.read.csv('dbfs:/FileStore/shared_uploads/kumaranonymous77@gmail.com/artist.csv', header=True, inferSchema=True)

df_canvas_size = spark.read.csv('dbfs:/FileStore/shared_uploads/kumaranonymous77@gmail.com/canvas_size.csv', header=True, inferSchema=True)

df_image_link = spark.read.csv('dbfs:/FileStore/shared_uploads/kumaranonymous77@gmail.com/image_link.csv', header=True, inferSchema=True)

df_museum = spark.read.csv('dbfs:/FileStore/shared_uploads/kumaranonymous77@gmail.com/museum-1.csv', header=True, inferSchema=True)

df_museum_hours = spark.read.csv('dbfs:/FileStore/shared_uploads/kumaranonymous77@gmail.com/museum_hours.csv', header=True, inferSchema=True)

df_product_size = spark.read.csv('dbfs:/FileStore/shared_uploads/kumaranonymous77@gmail.com/product_size.csv', header=True, inferSchema=True)

### Fetch all the paintings which are not displayed on any museums?


In [0]:
result = df_work.filter(df_work.museum_id.isNull())
result.show()

#----------------------------------------------------------------------------------------
# Through SQL approach\\\///converting dataframe to a temporary SQL Table
'''df_work.createOrReplaceTempView("work_table")
result = spark.sql("select * from work_table where museum_id is null")
result.show()'''

+-------+--------------------+---------+-------+---------+
|work_id|                name|artist_id|  style|museum_id|
+-------+--------------------+---------+-------+---------+
| 125752|Arabian Horses at...|      757|Baroque|     null|
| 125818|Count Halm on His...|      757|Baroque|     null|
| 125763|Napoleon Before t...|      757|Baroque|     null|
| 125774|Peasants Resting ...|      757|Baroque|     null|
| 125785|Portrait Oberleut...|      757|Baroque|     null|
| 125796|The Rescue of Cou...|      757|Baroque|     null|
| 125807|     The Stable Yard|      757|Baroque|     null|
|  24532|Jacob A. Stamler ...|      563|   null|     null|
| 124470| Kaleda off Le Havre|      563|   null|     null|
| 124479|R. Bell &amp; Co....|      563|   null|     null|
| 124488|Steam Sailing Shi...|      563|   null|     null|
| 124497|The American Ship...|      563|   null|     null|
| 124506|The Atalanta Runn...|      563|   null|     null|
| 124515|The Auxiliary Ste...|      563|   null|     nul

### Are there museuems without any paintings?



In [0]:
result_df = df_museum.alias("m").join(df_work.alias("w"), "museum_id", "left_outer").filter("w.museum_id IS NULL")
result_df.show()

+---------+----+-------+----+-----+------+-------+-----+---+-------+----+---------+-----+
|museum_id|name|address|city|state|postal|country|phone|url|work_id|name|artist_id|style|
+---------+----+-------+----+-----+------+-------+-----+---+-------+----+---------+-----+
+---------+----+-------+----+-----+------+-------+-----+---+-------+----+---------+-----+



### How many paintings have an asking price of more than their regular price? 

In [0]:
result_three = df_product_size.filter("sale_price > regular_price") 
result_three.show()

+-------+-------+----------+-------------+
|work_id|size_id|sale_price|regular_price|
+-------+-------+----------+-------------+
+-------+-------+----------+-------------+



###Identify the paintings whose asking price is less than 50% of its regular price

In [0]:
result_Four = df_product_size.filter("sale_price > regular_price*0.5") 
result_Four.show()

+-------+-------+----------+-------------+
|work_id|size_id|sale_price|regular_price|
+-------+-------+----------+-------------+
| 160228|     24|        85|           85|
| 160228|     30|        95|           95|
| 160236|     24|        85|           85|
| 160236|     30|        95|           95|
| 160244|     24|        85|           85|
| 160244|     30|        95|           95|
| 160252|     24|        85|           85|
| 160252|     30|        95|           95|
| 160260|     24|        75|           75|
| 160260|     30|        95|           95|
| 160268|     24|        85|           85|
| 160268|     30|        95|           95|
| 125752|     30|        95|           95|
| 125752|     24|        85|           85|
| 125752|   3024|       305|          535|
| 125752|   3226|       325|          575|
| 125752|   3629|       375|          675|
| 125752|   4030|       405|          735|
| 125752|   4836|       495|          915|
| 125752|   6048|       675|         1275|
+-------+--

### Which canva size costs the most?

In [0]:
# Defining window specification

window_spec = Window.orderBy(desc('sale_price'))


In [0]:
# Adding a rank column using the window specification
ranked_df = df_product_size.withColumn('rnk', rank().over(window_spec))

result_Five = ranked_df.join(df_canvas_size.withColumnRenamed('size_id', 'canvas_size_id'), 
                           on=(col('size_id') == col('canvas_size_id')) & (col('rnk') == 1)) \
                     .select('label', 'sale_price')


###Identify the museums with invalid city information

In [0]:
filtered_museum_df = df_museum.filter(col("city").rlike("^[0-9]"))


###Fetch the top 10 most famous painting subject

In [0]:
# Joining work_df and subject_df to get subject information for each painting

joined_df = df_work.join(df_subject, work_df.work_id == subject_df.work_id)
# Grouping by subject and counting the number of paintings for each subject
subject_counts_df = joined_df.groupBy("subject").agg(count("*").alias("no_of_paintings"))
# Defining a Window specification to rank subjects based on the number of paintings
window_spec = Window.orderBy(subject_counts_df["no_of_paintings"].desc())

# Adding a rank column based on the number of paintings for each subject
subject_ranked_df = subject_counts_df.withColumn("ranking", rank().over(window_spec))

# Selecting the top 10 most famous subjects
top_10_subjects_df = subject_ranked_df.filter(subject_ranked_df["ranking"] <= 10)


### Identifing museum name, city which are open on both Sunday and Monday

In [0]:
# Filter museum_hours_df for museums open on Sunday
sunday_museums_df = df_museum_hours.filter(df_museum_hours["day"] == "Sunday")

# Filter museum_hours_df for museums open on Monday
monday_museums_df = df_museum_hours.filter(df_museum_hours["day"] == "Monday")

joint_df = sunday_museums_df.join(monday_museums_df, "museum_id")
joint_df.show()
result_df = joint_df.join(df_museum, "museum_id") \
                    .select("name", "city") \
                    .distinct()
result_df.show()


+--------------------+------------+
|                name|        city|
+--------------------+------------+
|Solomon R. Guggen...|    New York|
|       Israel Museum|   Jerusalem|
|     Van Gogh Museum|   Amsterdam|
|  Mauritshuis Museum|    Den Haag|
|    The Prado Museum|      Madrid|
|The Barnes Founda...|Philadelphia|
|Museum of Fine Ar...|       Nancy|
|National Maritime...|      London|
|Pushkin State Mus...|      Moscow|
|    The Tate Gallery|      London|
|National Gallery ...|  Washington|
|The Museum of Mod...|    New York|
|     Musée du Louvre|       75001|
|  Museum of Grenoble|       38000|
|    National Gallery|      London|
|         Rijksmuseum|   Amsterdam|
|The Metropolitan ...|    New York|
|Los Angeles Count...| Los Angeles|
|National Gallery ...|   Melbourne|
|Nelson-Atkins Mus...| Kansas City|
+--------------------+------------+
only showing top 20 rows



### How many museums are open every single day?

In [0]:
result_df = df_museum_hours \
    .groupBy("museum_id") \
    .agg(count("*").alias("count")) \
    .filter("count = 7") \
    .select(count("*").alias("count"))
result_df.show()

+-----+
|count|
+-----+
|   18|
+-----+

